In [1]:
import mindsdb_sdk
import os
from dotenv import load_dotenv
import ozon3 as ooo

In [2]:
load_dotenv()

o3 = ooo.Ozon3(os.environ['API_KEY_OOO'])
server = mindsdb_sdk.connect()

In [3]:
def get_city():
    city = 'Mumbai'
    return city

In [42]:
data = o3.get_historical_data(city = 'mumbai')
data = data.dropna()

d:\Siddhant\Open Source\AQI Prediction\venv\lib\site-packages\ozon3\ozon3.py:572: UserWarning: city_id was not supplied. Searching for "mumbai" yields city ID 7020 with station name "['Mumbai US Consulate, India']", with country code "IN". Ozon3 will return air quality data from that station. If you know this is not the correct city you intended, you can use get_city_station_options method first to identify the correct city ID.
  warnings.warn(


In [43]:
def historic_data():
    city = get_city()
    data = o3.get_historical_data(city = city)
    data = data.dropna()
    return data
historic_data()

d:\Siddhant\Open Source\AQI Prediction\venv\lib\site-packages\ozon3\ozon3.py:572: UserWarning: city_id was not supplied. Searching for "Mumbai" yields city ID 7020 with station name "['Mumbai US Consulate, India']", with country code "IN". Ozon3 will return air quality data from that station. If you know this is not the correct city you intended, you can use get_city_station_options method first to identify the correct city ID.
  warnings.warn(


,date,pm2.5
0,2024-07-16,58.0
1,2024-07-15,47.0
2,2024-07-14,46.0
3,2024-07-13,59.0
4,2024-07-12,69.0
...,...,...
3501,2014-12-15,139.0
3502,2014-12-14,147.0
3503,2014-12-13,184.0
3504,2014-12-12,191.0


In [28]:
try:
    project = server.get_project('aqi')
except:
    project = server.create_project('aqi')

In [35]:
def add_data():
    data = historic_data()
    try:
        files_db = server.get_database('files')
        files_db.create_table('mumbai_data', data)
        print('Data added')
    except:
        table = files_db.get_table('mumbai_data')
        print('file table already exists')
    return table

add_data()

d:\Siddhant\Open Source\AQI Prediction\venv\lib\site-packages\ozon3\ozon3.py:572: UserWarning: city_id was not supplied. Searching for "Mumbai" yields city ID 7020 with station name "['Mumbai US Consulate, India']", with country code "IN". Ozon3 will return air quality data from that station. If you know this is not the correct city you intended, you can use get_city_station_options method first to identify the correct city ID.
  warnings.warn(


file table already exists


Table(files.mumbai_data)

In [23]:
def create_models():
    try:
        project = server.get_project('aqi')
        print('Project exists')
    except:
        project = server.create_project('aqi')
        print('Project created')

    try:
        model = project.models.create(
    name='city_aqi_forecast_model',
    query='SELECT date AS ds, `pm2.5` AS y FROM files.mumbai_data',
    predict='y',
    timeseries_options={
        'order': 'ds',
        'horizon': 7
    },
    engine='timegpt_engine',
    timegpt_api_key=os.environ['API_KEY_TIMEGPT']
)
        print('Model created')
    except Exception as e:
        model = project.models.get('city_aqi_forecast_model')
        print('Model already exists')

    return model

create_models()

Project exists
Model created


Model(city_aqi_forecast_model, status=generating)

In [25]:
def prediction(project):
    model = create_models()
    try:
        pred = model.predict({
            'ds' : '2024-07-15'
        })
        print('prediction executed:', pred)
    except Exception as e:
        print('Error:', e)

prediction(project)

Project exists
Model already exists
Error: INTERNAL SERVER ERROR: {"message": "MLEngineException: [timegpt_engine/city_aqi_forecast_model]: KeyError: \"['y'] not in index\""}


In [62]:
data = o3.get_historical_data(city='Houston')
data = data[['date','pm2.5']]
try:
    files_db = server.get_database('files')
    files_db.create_table('city_data', data)
    print('Data added')
except Exception as e:
    table = files_db.get_table('city_data')
    print('file table already exists and error:', e)

d:\Siddhant\Open Source\AQI Prediction\venv\lib\site-packages\ozon3\ozon3.py:572: UserWarning: city_id was not supplied. Searching for "Houston" yields city ID 203 with station name "['Houston Aldine, Houston, Texas']", with country code "US". Ozon3 will return air quality data from that station. If you know this is not the correct city you intended, you can use get_city_station_options method first to identify the correct city ID.
  warnings.warn(


file table already exists and error: BAD REQUEST: {"title": "File already exists", "detail": "File with name 'city_data' already exists"}


In [60]:
def create_model():

  project = server.get_project('aqi')

  model = project.models.create(
    name='aqi_forecast_model',
    predict='`pm2.5`',
    query='SELECT * FROM files.city_data',
    timeseries_options={
        'order': 'date',
        'horizon': 5
    },
    engine='timegpt',
    timegpt_api_key='nixtla-tok-ojwrR4rPpifERgb2RSrCjPx1Io6hh11YOmenD1ulLx8GogNe4AJHFwpK52kKpgMEedK7DcEAn8OlDOYV'
)
  return model


In [61]:
def get_aqi_predictions(project, limit=10):
    query = f"""
    SELECT m.date, m.`pm2.5` AS predicted_pm25, t.`pm2.5` AS actual_pm25 
    FROM files.city_data t 
    JOIN aqi_forecast_nixtla m
    WHERE t.date > LATEST
    LIMIT {limit}
    """
    return project.query(query)

project = server.get_project('aqi')

results = get_aqi_predictions(project)
df = results.fetch()

print(df)

RuntimeError: [timegpt/aqi_forecast_nixtla]: ValueError: Need at least 3 dates to infer frequency

In [48]:
def get_aqi_predictions(project, limit=10):
    query = f"""
    SELECT m.date, m.`pm2.5` AS predicted_pm25, t.`pm2.5` AS actual_pm25 
    FROM files.historic_houston t 
    JOIN aqi_forecast_nixtla m
    WHERE t.date > LATEST
    LIMIT {limit}
    """
    return project.query(query)

project = server.get_project('aqi')

results = get_aqi_predictions(project)
df = results.fetch()

print(df)

                         date  predicted_pm25  actual_pm25
0  2024-07-04 00:00:00.000000       57.626740    65.000000
1  2024-07-05 00:00:00.000000       55.571793    65.000000
2  2024-07-06 00:00:00.000000       56.068810    36.647357
3  2024-07-07 00:00:00.000000       55.382233    39.000000
4  2024-07-08 00:00:00.000000       51.697842    40.000000


In [50]:
def get_aqi_city_predictions(project, limit=10):
    
    query = f"""
    SELECT m.date, m.`pm2.5` AS predicted_pm25, t.`pm2.5` AS actual_pm25 
FROM files.mumbai_data t 
JOIN aqi_forecast_nixtla m
WHERE t.date > LATEST
LIMIT 10;
    """
    return project.query(query)

project = server.get_project('aqi')

results = get_aqi_city_predictions(project)
tf = results.fetch()

print(tf)

RuntimeError: [timegpt/aqi_forecast_nixtla]: Exception: Your target variable contains NA, please check

In [57]:
def create_model():

  project = server.get_project('aqi')

  model = project.models.create(
    name='aqi_forecast_model',
    predict='`pm2.5`',
    query='SELECT * FROM files.city_data',
    timeseries_options={
        'order': 'date',
        'horizon': 5
    },
    engine='timegpt',
    timegpt_api_key='nixtla-tok-ojwrR4rPpifERgb2RSrCjPx1Io6hh11YOmenD1ulLx8GogNe4AJHFwpK52kKpgMEedK7DcEAn8OlDOYV'
)
  return model

create_model()

Model(aqi_forecast_mum, status=generating)

In [58]:
results = project.query("""
    SELECT m.date, m.`pm2.5` AS predicted_pm25, t.`pm2.5` AS actual_pm25 
    FROM files.mumbai_data t 
    JOIN aqi_forecast_mum m
    WHERE t.date > LATEST
    LIMIT 10
""")


predictions = results.fetch()
print(predictions)

RuntimeError: [timegpt/aqi_forecast_mum]: Exception: Your target variable contains NA, please check